- Get hugging face token here https://huggingface.co/settings/tokens
- Click agree in pyannote projects including [Segmentation](https://huggingface.co/pyannote/segmentation) , [Voice Activity Detection (VAD)](https://huggingface.co/pyannote/voice-activity-detection), and [Speaker Diarization](https://huggingface.co/pyannote/speaker-diarization)


- 在这里获取你的 Hugging Face token https://huggingface.co/settings/tokens
- 在这些项目中同意协议：[Segmentation](https://huggingface.co/pyannote/segmentation) , [Voice Activity Detection (VAD)](https://huggingface.co/pyannote/voice-activity-detection), [Speaker Diarization](https://huggingface.co/pyannote/speaker-diarization)

In [4]:
RAW_DIR = "./raw" # the foler of the source video/audio file
OUTPUT_DIR = "./out" # the folder where the output will be placed
MIN_SPEAKERS = 1
MAX_SPEAKERS = 7
CHUNK_SIZE = 10 # lower chunk_size will make each sentence shorter
diarize = True

# disabled
# vad_onset = 0.05
# vad_offset = 0

diarize_param = "--diarize" if diarize else ""

In [2]:
import os, dotenv
dotenv.load_dotenv()

hf_token = os.environ.get("HF_TOKEN", None) or "fill your hf token here/将此字符串替换为你的hf token"
raw_file = "u149.mp4"
openai_key = os.environ.get("OPENAI_KEY", None) or "fill your openai key here/将此字符串替换为你的openai key"
api_base = os.environ.get("API_BASE", None) or "https://api.openai.com/v1" # replace with your proxy


In [7]:

!python -m whisperx --model large-v2 --language ja {RAW_DIR}/{raw_file} --hf_token {hf_token} --output_dir {OUTPUT_DIR} --align_model jonatasgrosman/wav2vec2-large-xlsr-53-japanese --max_line_width 30 --min_speakers {MIN_SPEAKERS} --max_speakers {MAX_SPEAKERS}  {diarize_param} --chunk_size {CHUNK_SIZE} --output_format json


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.0.8. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../.cache/torch/whisperx-vad-segmentation.bin`
Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.0. Bad things might happen unless you revert torch to 1.x.
>>Performing transcription...
>>Performing alignment...
>>Performing diarization...
Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.0.8. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`
Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert p

In [36]:
import pysubs2, json
from typing import cast
from better_n46_whisper.TYPES import Segment

ext = os.path.splitext(raw_file)[1]

with open(f"{OUTPUT_DIR}/{os.path.basename(raw_file).replace(ext, '.json')}") as f:
    result = json.load(f)

subs = pysubs2.load_from_whisper(result)
subs.save(f"{OUTPUT_DIR}/{os.path.basename(raw_file).replace(ext, '.ass')}")

In [2]:
texts_by_speaker: dict[str, set] = {}
for segment in result["segments"]:
    segment = cast(Segment, segment)
    if segment["speaker"] is None:
        continue
    texts_by_speaker.setdefault(segment["speaker"], set())
    texts_by_speaker[segment["speaker"]].add(segment["text"])
    
style = subs.styles["Default"]
style.fontname = "思源黑体 CN Medium"
style.fontsize = 72
# style.primarycolor = "&H00FFFFFF"
# style.secondarycolor = "&H000000FF"
# style.outlinecolor = "&H00000000"
# style.backcolor = "&H00000000"
style.bold = True
# style.scalex = 100
# style.scaley = 100
# style.spacing = 0
# style.angle = 0
# style.borderstyle = 1
# style.outline = 2
# style.shadow = 2
# # style.alignment = 2
# style.marginl = 10
# style.marginr = 10
# style.marginv = 10
# style.encoding = 1

for speaker in texts_by_speaker.keys():
    subs.styles[speaker] = subs.styles["Default"].copy()

# subs.styles["SPEAKER_02"].primarycolor = Color(194, 225, 137)
# subs.styles["SPEAKER_03"].primarycolor = Color(92, 136, 218)

for i in range(len(subs)):
    speaker = result["segments"][i]["speaker"]
    subs[i].style = speaker

In [5]:
from better_n46_whisper.translator.base import LLMTranslator

translator = LLMTranslator(openai_key, temperature=0.5, model="gpt-3.5-turbo-16k", api_base=api_base)
translator.translate_ass(subs, batch_size=30)
subs.save(f"{OUTPUT_DIR}/{os.path.basename(raw_file).replace(ext, '-translated.ass')}")